In [1]:
!pip install lifelines
!pip install pyckmeans
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 10.9 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=94ccc92ec8b483e623b1946e306d2c720c2f240160415efcf084511d52335312
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 898.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyckmeans: filename=pyckmeans-0.9.4-cp310-cp310-linux_x86_64.whl size=93216 sha256=555ed75678071a04c53e769bfe57b4f197273ee98c59dde00696ed29aad939d6
  Stored in directory: /root/.cache/pip/wheels/c3/83/bb/d0fc31d29f361aff1f400ca2fb2d32c98f900e1aa480229f99
Successfully built pyckmeans
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.7 MB

In [2]:
import numpy as np
import pandas as pd
from pyckmeans import CKmeans
import matplotlib.pyplot as plt

from lifelines.fitters.coxph_fitter import CoxPHFitter

from google.colab import drive

import wandb
import os

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# --- Load raw BRCA data ---
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/TCGA-BRCA_1079.zip

Mounted at /content/gdrive
Archive:  gdrive/MyDrive/TCGA-BRCA_1079.zip
  inflating: TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv  


In [5]:
def drop_elements_from_cluster(cluster, consensus_matrix):
  new_cluster = []
  for j in range(0, len(cluster)):
    if consensus_matrix[cluster[0]][cluster[j]] == 1:
      new_cluster.append(cluster[j])
  return new_cluster

In [6]:
def convert_indices_to_binary_list(number_elements, cluster_0, cluster_1, cluster_2):
  cluster0_column = []
  cluster1_column = []
  cluster2_column = []
  for k in range(0, number_elements):
    if k in cluster_0:
      cluster0_column.append(1)
      cluster1_column.append(0)
      cluster2_column.append(0)
    elif k in cluster_1:
      cluster0_column.append(0)
      cluster1_column.append(1)
      cluster2_column.append(0)
    elif k in cluster_2:
      cluster0_column.append(0)
      cluster1_column.append(0)
      cluster2_column.append(1)
    else:
      cluster0_column.append(0)
      cluster1_column.append(0)
      cluster2_column.append(0)

  return [cluster0_column, cluster1_column, cluster2_column]

In [7]:
def fit_cox_model(cluster, data_frame):
  print("Original dataframe shape: ", data_frame.shape)
  cluster_frame = pd.DataFrame({'cluster': cluster})
  # Concatenate cluster membership to data table
  if 'cluster' in data_frame.columns.values:
    data_frame = data_frame.drop('cluster', axis = 1) # Drop columns that could have been created previously
  data_frame = pd.concat([data_frame, cluster_frame], axis = 1)

  # Get rows with null entry
  null_mask = data_frame.isnull().any(axis=1)
  null_rows = data_frame[null_mask]
  print(null_rows)

  # Remove rows with null entry
  data_frame.dropna(inplace=True) # remove all rows with any null value
  print("Final dataframe shape: ", data_frame.shape)

  # Fit Cox model
  cph = CoxPHFitter()
  cph.fit(data_frame, duration_col = 'OS.time', event_col = 'OS')
  cph.print_summary()

  # exp(coef) <=> hazard ratio // AN ESTIMATE OF THE TRUE HAZARD RATIO. IT HAS A STANDARD ERROR ASSOCIATED WITH IT.
  # A one unit increase in the covariate will increase the hazard by the hazard ratio

  # Every coefficient comes with a p-value
  # The p-value represents the probability of observing this coefficient in a sample if the null hypothesis was true.
  # The null hypothesis states that the coefficient=0, meaning that the predictor variable does not influence the hazard rate (occurence of the event)

  # The CoxPHFitter computes p-values using the chi-squared test.
  # The reference is in "Survival Analysis by John P. Klein and Melvin L. Moeschberger, Second Edition", page 256

  p_values = cph._compute_p_values()
  hazard_ratios = cph.hazard_ratios_.tolist()
  coefficients_ci = cph.confidence_intervals_
  print(p_values)
  print(hazard_ratios)
  print(coefficients_ci)
  print("p-value:", p_values[4])
  print("hazard-ratio:", hazard_ratios[4])

  return p_values[4], hazard_ratios[4]

In [16]:
for i in range (4, 5):
  data_frame = pd.read_csv("TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv", sep="\t")
  data_frame = data_frame[['age', 'OS', 'OS.time', 'stage_2', 'stage_3', 'stage_4']]

  name_embedding = "VAE_Z_EM_Mean_{i}".format(i=i)
  #name_embedding = "UMAP_EM"
  embeddings = np.loadtxt(name_embedding+".txt")
  print("Embeddings shape: ", embeddings.shape)
  print("------------- i =", i, "--------------------")
  number_of_clusters = 3
  ckm = CKmeans(k=number_of_clusters, n_rep=20, p_samp=1, p_feat=1)
  ckm.fit(embeddings)
  ckm_results = ckm.predict(embeddings, linkage_type='average')
  print(ckm_results.cl)
  consensus_matrix = ckm_results.cmatrix
  print(consensus_matrix)

  cluster_0 = []
  cluster_1 = []
  cluster_2 = []
  for j in range(0, ckm_results.cl.shape[0]):
    if ckm_results.cl[j] == 1:
      cluster_1.append(j)
    elif ckm_results.cl[j] == 2:
      cluster_2.append(j)
    elif ckm_results.cl[j] == 0:
      cluster_0.append(j)
  #print(cluster_0)
  #print(cluster_1)
  #print(cluster_2)
  print("Cluster sizes: ", len(cluster_0), ", ", len(cluster_1), ", ", len(cluster_2))
  print("Sum of cluster sizes: ", len(cluster_0)+len(cluster_1)+len(cluster_2))

  cluster_0 = drop_elements_from_cluster(cluster_0, consensus_matrix)
  cluster_1 = drop_elements_from_cluster(cluster_1, consensus_matrix)
  cluster_2 = drop_elements_from_cluster(cluster_2, consensus_matrix)
  print("Cluster sizes after dropping:", len(cluster_0), ", ", len(cluster_1), ", ", len(cluster_2))

  cluster_column_list = convert_indices_to_binary_list(embeddings.shape[0], cluster_0, cluster_1, cluster_2)
  print("Column_lengths: ",  len(cluster_column_list[0]))

  for j in range(0, number_of_clusters):
    if cluster_column_list[j].count(1) > 10:
      print("Cluster size in column form:", cluster_column_list[j].count(1))
      p_value, hazard_ratio = fit_cox_model(cluster_column_list[j], data_frame)
      print("p-value:", p_value)
      print("hazard ratio:", hazard_ratio)
      cluster_name = "{base}_{j}".format(base=name_embedding, j=j)
      run = wandb.init(project="SURVIVAL_ANALAYSIS_k=3", # Set the project where this run will be logged
                       name=cluster_name
      )

      wandb.log({
                "p_value": p_value,
                "hazard_ratio": hazard_ratio,
                "cluster_size": cluster_column_list[j].count(1)
            })

      if p_value < 0.0012:
        file_name = cluster_name + ".txt"
        np.savetxt(file_name, cluster_column_list[j])



Embeddings shape:  (1079, 10)
------------- i = 4 --------------------
[2 2 2 ... 2 2 2]
[[1.   0.35 0.8  ... 0.7  0.7  0.5 ]
 [0.35 1.   0.25 ... 0.25 0.35 0.65]
 [0.8  0.25 1.   ... 0.8  0.8  0.4 ]
 ...
 [0.7  0.25 0.8  ... 1.   0.6  0.6 ]
 [0.7  0.35 0.8  ... 0.6  1.   0.2 ]
 [0.5  0.65 0.4  ... 0.6  0.2  1.  ]]
Cluster sizes:  147 ,  284 ,  648
Sum of cluster sizes:  1079
Cluster sizes after dropping: 36 ,  2 ,  1
Column_lengths:  1079
Cluster size in column form: 36
Original dataframe shape:  (1079, 6)
       age   OS  OS.time  stage_2  stage_3  stage_4  cluster
0     55.0  0.0   4047.0      NaN      NaN      NaN        0
180   81.0  0.0    608.0      NaN      NaN      NaN        0
213   76.0  0.0   1217.0      NaN      NaN      NaN        0
222   76.0  0.0    304.0      NaN      NaN      NaN        0
223   40.0  0.0    304.0      NaN      NaN      NaN        0
225   69.0  0.0     31.0      NaN      NaN      NaN        0
235   68.0  0.0    579.0      NaN      NaN      NaN        0

<lifelines.CoxPHFitter: fitted with 1054 total observations, 915 right-censored observations>
             duration col = 'OS.time'
                event col = 'OS'
      baseline estimation = breslow
   number of observations = 1054
number of events observed = 139
   partial log-likelihood = -745.33
         time fit was run = 2024-03-08 19:40:13 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.04       1.04       0.01             0.02             0.05                 1.02                 1.05
stage_2     0.59       1.81       0.28             0.04             1.15                 1.04                 3.14
stage_3     1.35       3.86       0.30             0.77             1.93                 2.16                 6.90
stage_4     2.46      11.70       0.37             1.74             3.18                 5.69                24.04
cluster     1.30       3.68       0.40             0.53             2.08                 1.69                 7.98

            cmp to    z      p   -log2(p)
covariate                                
age           0.00 5.40 <0.005      23.87
stage_2       0.00 2.12   0.03       4.86
stage_3       0.00 4.57 <0.005      17.62
stage_4       0.00 6.69 <0.005      35.39
cluster       0.00 3.29 <0.005       9.98
---
Concordance = 0.76
Partial AIC = 1500.66
log-likelihood ratio test = 83.26 on 5 df
-log2(p) of ll-ratio test = 52.35

[6.52282767e-08 3.44111564e-02 4.97301023e-06 2.22319560e-11
 9.90575674e-04]
[1.0376441146927993, 1.812409817938755, 3.862388852939553, 11.697545560588978, 3.676646405466848]
           95% lower-bound  95% upper-bound
covariate                                  
age               0.023550         0.050356
stage_2           0.043648         1.145667
stage_3           0.771234         1.931338
stage_4           1.738917         3.179841
cluster           0.527106         2.076896
p-value: 0.0009905756738502187
hazard-ratio: 3.676646405466848
p-value: 0.0009905756738502187
hazard ratio: 3.676646405466848


cluster_size,▁
hazard_ratio,▁
p_value,▁
cluster_size,31
hazard_ratio,2.82474
p_value,0.0246
